In [9]:
import numpy as np

from PIL import Image
from pathlib import Path

from src.utils import Picture

input_dir = Path.cwd() / "data" / "raw"
output_dir = Path.cwd() / "data" / "output"

choose_pic = Picture.ISLAND
inp = Image.open(f"{input_dir}/{choose_pic.value}")
img = np.array(inp, dtype='uint8')

In [4]:
"""
Steps of algorithm:
1. Calculate H' and W':
    W' = |Wcosθ| + |Hsinθ|
    H' = |Wsinθ| + |Hcosθ|
2. Create the empty np.arr H' * W' * 3;
3. Iterate every element in new arr;
4. For every element (y', x') do InvMapping, get (y, x);
5. If (y, x) out of border:
    return fill_value
    else:
    do BilInt. Check it neighbours and calculate new channel
6. Change dst[y', x'] with result of BilInt;
7. Return to step 3;

"""

"\nSteps of algorithm:\n1. Calculate H' and W':\n    W' = |Wcosθ| + |Hsinθ|\n    H' = |Wsinθ| + |Hcosθ|\n2. Create the empty np.arr H' * W' * 3;\n3. Iterate every element in new arr;\n4. For every element (y', x') do InvMapping, get (y, x);\n5. If (y, x) out of border:\n    return fill_value\n    else:\n    do BilInt. Check it neighbours and calculate new channel\n6. Change dst[y', x'] with result of BilInt;\n7. Return to step 3;\n\n"

In [5]:
def invmap_bilint(y_dst: int, x_dst: int, theta: float, src: np.ndarray, dst: np.ndarray):
    # размер src и dst
    h, w, _ = src.shape
    nh, nw, _ = dst.shape

    # центр src и dst
    cx, cy = w / 2, h / 2
    ncx, ncy = nw / 2, nh / 2

    # сдвигаем x_dst и y_dst так, чтобы ncx, ncy стали центром координат
    xd = x_dst - ncx
    yd = y_dst - ncy

    # математика
    xs = xd * np.cos(theta) + yd * np.sin(theta)
    ys = -xd * np.sin(theta) + yd * np.cos(theta)

    # получили координаты в x,y в src со сдвигом, возвращаем на свое место
    x_src = xs + cx
    y_src = ys + cy

    fill_value = np.array([128, 128, 128], dtype=src.dtype)
    
    x0, y0 = int(np.floor(x_src)), int(np.floor(y_src))
    x1, y1 = x0 + 1, y0 + 1

    # проверка на oob (out of border)
    if (
        x0 < 0 or x1 >= w or
        y0 < 0 or y1 >= h
    ):
        return fill_value

    Q00 = (y0, x0)
    Q01 = (y0, x1)
    Q10 = (y1, x0)
    Q11 = (y1, x1)

    dx = x_src - x0 
    dy = y_src - y0

    result = (
        (((1 - dx) * (1 - dy)) * src[Q00]) +
        ((dx * (1 - dy)) * src[Q01]) + 
        (((1 - dx) * dy) * src[Q10]) +
        ((dx * dy) * src[Q11])
    )

    return result


In [6]:
# def bil_int(y_src: float, x_src: float, src: np.ndarray):
#     h, w, _ = src.shape
#     fill_value = np.array([128, 128, 128], dtype=src.dtype)
    
#     x0, y0 = int(np.floor(x_src)), int(np.floor(y_src))
#     x1, y1 = x0 + 1, y0 + 1

#     # проверка на oob (out of border)
#     if (
#         x0 < 0 or x1 >= w or
#         y0 < 0 or y1 >= h
#     ):
#         return fill_value

#     Q00 = (y0, x0)
#     Q01 = (y0, x1)
#     Q10 = (y1, x0)
#     Q11 = (y1, x1)

#     dx = x_src - x0 
#     dy = y_src - y0

#     result = (
#         (((1 - dx) * (1 - dy)) * src[Q00]) +
#         ((dx * (1 - dy)) * src[Q01]) + 
#         (((1 - dx) * dy) * src[Q10]) +
#         ((dx * dy) * src[Q11])
#     )

#     return result 





In [ ]:
from numba import njit
import numpy as np


@njit(fastmath=True)
def rotate_numba(src, theta):
    h, w, c = src.shape

    cos_t = np.cos(theta)
    sin_t = np.sin(theta)

    nh = int(np.ceil(abs(h * cos_t) + abs(w * sin_t)))
    nw = int(np.ceil(abs(h * sin_t) + abs(w * cos_t)))

    dst = np.zeros((nh, nw, c), dtype=src.dtype)

    cx = w / 2.0
    cy = h / 2.0
    ncx = nw / 2.0
    ncy = nh / 2.0

    for y in range(nh):
        for x in range(nw):

            # inverse mapping
            xd = x - ncx
            yd = y - ncy

            xs =  xd * cos_t + yd * sin_t
            ys = -xd * sin_t + yd * cos_t

            x_src = xs + cx
            y_src = ys + cy

            x0 = int(np.floor(x_src))
            y0 = int(np.floor(y_src))
            x1 = x0 + 1
            y1 = y0 + 1

            if x0 < 0 or x1 >= w or y0 < 0 or y1 >= h:
                for ch in range(c):
                    dst[y, x, ch] = 128
                continue

            dx = x_src - x0
            dy = y_src - y0

            w00 = (1 - dx) * (1 - dy)
            w01 = dx * (1 - dy)
            w10 = (1 - dx) * dy
            w11 = dx * dy

            for ch in range(c):
                dst[y, x, ch] = (
                    w00 * src[y0, x0, ch] +
                    w01 * src[y0, x1, ch] +
                    w10 * src[y1, x0, ch] +
                    w11 * src[y1, x1, ch]
                )

    return dst

theta = np.deg2rad(144)
out = rotate_numba(img, theta)
Image.fromarray(out).save(f"{output_dir}/{choose_pic.value} - rotated to 66 deg with numba.jpg")

In [8]:
# h, w, c = img.shape
# a = 144
# theta = np.deg2rad(a)

# # Матрица поворота
# R_inv = [
#     [np.cos(theta), np.sin(theta)],
#     [-np.sin(theta), np.cos(theta)]
# ]

# # Размеры нового холста
# nh = int(np.ceil(abs(h * np.cos(theta)) + abs(w * np.sin(theta))))
# nw = int(np.ceil(abs(h * np.sin(theta)) + abs(w * np.cos(theta))))

# # Создание холста
# nimg = np.zeros((nh, nw, c), dtype=np.uint8)

# from tqdm import tqdm

# for y in tqdm(range(nh), desc="Rotate rows"):
#     for x in range(nw):
#         px= invmap_bilint(y, x, theta, img, nimg)
#         nimg[y, x] = px.astype(np.uint8)

Rotate rows:   9%|▊         | 851/9803 [00:26<04:36, 32.39it/s]


KeyboardInterrupt: 

In [ ]:
Image.fromarray(nimg).save(f"{output_dir}/{choose_pic.value} - rotated to {a} deg.jpg")
